<center><img src="./images/nup_logo_dark.jpeg" width=300 style="display: inline-block;"></center>

## Advanced ML
### Topic modeling and word2vec

<br />
April 8, 2025


This notebook examines two topic modeling models from the `gensim` library:
  - LDA (Latent Dirichlet Allocation)
  - word2vec

Sources of inspiration:
  - https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
  - https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

### LDA (Latent Dirichlet Allocation)
We install the topic modeling library gensim (http://radimrehurek.com/gensim/) and load the NLTK library (http://nltk.org/), which will be needed for lemmatization.


In [1]:
!pip install --upgrade gensim
!pip install --upgrade nltk


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import nltk
import numpy as np
import os
import sys

from gensim import corpora, models, similarities
from math import log
from time import time

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

We read the collection of source texts into a list of documents. Each document is a list of lemmas (tokens). In this example, we load the entire collection into memory. In fact, `gensim` allows you to avoid this at all stages of model building.

The collection used is articles from the NeurIPS conference, one of the standard collections for topic modeling. The number of documents is about 1700, with each document having a length of 1000-2000 words. 

In [3]:
import tarfile
import re
import urllib.request, zipfile


tarfile_url = 'https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'
filename = 'nips12raw_str602.tgz'
# urllib.request.urlretrieve(tarfile_url, filename)

def extract_documents(fname=filename):
    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')


In [4]:
docs = list(extract_documents())
print(len(docs))
print(print(docs[0][:500]))

1740
1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a pr
None


Data preparation:
- Create a dictionary
- Perform lemmatization
- Build n-grams
- Filter out tokens that are too frequent or too rare

In [5]:
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

In [6]:
print(np.sum([len(doc) for doc in docs]))

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

print(np.sum([len(doc) for doc in docs]))

5461201
5115888


In [7]:
print(docs[1][:50])

['stochastic', 'learning', 'networks', 'and', 'their', 'electronic', 'implementation', 'joshua', 'alspector', 'robert', 'b', 'allen', 'victor', 'hut', 'and', 'srinagesh', 'satyanarayana', 'bell', 'communications', 'research', 'morristown', 'nj', 'abstract', 'we', 'describe', 'a', 'family', 'of', 'learning', 'algorithms', 'that', 'operate', 'on', 'a', 'recurrent', 'symmetrically', 'connected', 'neuromorphic', 'network', 'that', 'like', 'the', 'boltzmann', 'machine', 'settles', 'in', 'the', 'presence', 'of', 'noise']


In [8]:
# Remove words that are only one character
docs = [[token for token in doc if len(token) > 1] for doc in docs]
print(np.sum([len(doc) for doc in docs]))

4629808


In [9]:
# Remove words with underscores, since we are going to use them as delimiters in bigrams
docs = [[token for token in doc if '_' not in token] for doc in docs]
print(np.sum([len(doc) for doc in docs]))

4626035


In [10]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Aleksandr.Avdiushenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
print(lemmatizer.lemmatize('abstracts'),
      lemmatizer.lemmatize('fishes'))

abstract fish


In [12]:
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [13]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)


2025-04-01 14:02:03,504 : INFO : collecting all words and their counts
2025-04-01 14:02:03,505 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2025-04-01 14:02:05,814 : INFO : collected 1114271 token types (unigram + bigrams) from a corpus of 4626035 words and 1740 sentences
2025-04-01 14:02:05,815 : INFO : merged Phrases<1114271 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
2025-04-01 14:02:05,815 : INFO : Phrases lifecycle event {'msg': 'built Phrases<1114271 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000> in 2.31s', 'datetime': '2025-04-01T14:02:05.815245', 'gensim': '4.3.3', 'python': '3.12.6 (v3.12.6:a4a2d2b0d85, Sep  6 2024, 16:08:03) [Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-15.3.2-arm64-arm-64bit', 'event': 'created'}


In [14]:
for token in bigram[docs[0][:100]]:
    if '_' in token:
        print(token)

abu_mostafa
california_institute
technology_pasadena
ca_abstract
neural_network
boolean_function
can_be
very_low
learning_rule
lower_bound


In [15]:
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document
            docs[idx].append(token)

In [16]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents
dictionary = Dictionary(docs)

2025-04-01 14:02:18,185 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2025-04-01 14:02:19,119 : INFO : built Dictionary<77939 unique tokens: ['0a', '2h', '2h2', '2he', '2n']...> from 1740 documents (total 4944995 corpus positions)
2025-04-01 14:02:19,119 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<77939 unique tokens: ['0a', '2h', '2h2', '2he', '2n']...> from 1740 documents (total 4944995 corpus positions)", 'datetime': '2025-04-01T14:02:19.119792', 'gensim': '4.3.3', 'python': '3.12.6 (v3.12.6:a4a2d2b0d85, Sep  6 2024, 16:08:03) [Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-15.3.2-arm64-arm-64bit', 'event': 'created'}


Remove words that are too rare (e.g., typos) and words that are too frequent (e.g., stop words or just common non-topic terms). The `filter_extremes` function removes tokens from the dictionary that appear in less than `no_below` documents or in more than `no_above` fraction of the total number of documents.

In [17]:
# Remove rare and common tokens
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2025-04-01 14:02:27,849 : INFO : discarding 69316 tokens: [('0a', 19), ('2h', 16), ('2h2', 1), ('2he', 3), ('a', 1740), ('about', 1058), ('abstract', 1740), ('after', 1087), ('alently', 2), ('all', 1658)]...
2025-04-01 14:02:27,849 : INFO : keeping 8623 tokens which were in no less than 20 and no more than 870 (=50.0%) documents
2025-04-01 14:02:27,860 : INFO : resulting dictionary: Dictionary<8623 unique tokens: ['2n', 'a2', 'a_follows', 'ability', 'abu']...>


Represent all documents in vector form (Bag-of-Words)

In [18]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [19]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8623
Number of documents: 1740


In [20]:
print(corpus[0][:10])

[(0, 4), (1, 1), (2, 1), (3, 2), (4, 4), (5, 4), (6, 1), (7, 1), (8, 1), (9, 1)]


### Training
Now we are ready to build a topic model for our collection. We will build an online LDA model, implemented in the `gensim` library. We specify the vectorized corpus of texts, the dictionary, and the number of topics (10). We will discuss the remaining parameters later.

In [21]:
start = time()
# Set training parameters.
num_topics = 10
chunksize = 2000  # batch-size
epochs = 5   
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=epochs,
    eval_every=eval_every
)
print('Evaluation time: {}'.format((time()-start) / 60))

2025-04-01 14:02:56,902 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2025-04-01 14:02:56,903 : INFO : using serial LDA version on this node
2025-04-01 14:02:56,908 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 1740 documents, updating model once every 1740 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2025-04-01 14:02:56,908 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2025-04-01 14:02:56,908 : INFO : PROGRESS: pass 0, at document #1740/1740
2025-04-01 14:03:02,186 : INFO : optimized alpha [0.058868334, 0.074743286, 0.06977449, 0.1018687, 0.086200245, 0.076768756, 0.06202958, 0.045915615, 0.10657893, 0.06756638]
2025-04-01 14:03:02,189 : INFO : topic #7 (0.046): 0.004*"tree" + 0.004*"node" + 0.003*"matrix" + 0.003*"cell" + 0.003*"image

Evaluation time: 0.25741286675135294


Let's see what we got. We are interested in part of the Phi matrix – the probabilities of words in topics. The NeurIPS collection is entirely dedicated to machine learning. It's difficult to evaluate the topics, though some interpretability can be traced.

In [22]:
for position in range(10):
    row = []
    for topic in range(10):
        row.append(model.show_topic(topic)[position][0].center(11, ' '))
    print(''.join(row))

   action      node     gradient   gaussian    field      neuron     image     distance    speech      rule   
   policy     bound   generalizationapproximation   visual      cell   recognition    node      signal     class   
reinforcement   class     dynamic    density      cell     circuit     object     graph     mixture   prediction
  optimal     hidden      net       matrix    stimulus    signal     layer      robot    component   control  
reinforcement_learning    tree     minimum   stochastic   image      spike      hidden      tree   recognition classifier
   reward   threshold     loss      prior      layer     response     face     tangent      word   classification
  control    theorem    optimal    bayesian   response  frequency     net      mapping     hidden    estimate 
  decision     net      solution    sample     memory    synaptic  character    matrix     layer   training_set
   neuron      let      teacher    sequence orientation    chip       view      image     

In [23]:
top_topics = model.top_topics(corpus)

2025-04-01 14:03:12,513 : INFO : CorpusAccumulator accumulated stats from 1000 documents


In [24]:
top_topics[0]

([(0.008333934, 'field'),
  (0.007421196, 'visual'),
  (0.0066706464, 'cell'),
  (0.0065103243, 'stimulus'),
  (0.005340202, 'image'),
  (0.0052392543, 'layer'),
  (0.0051620053, 'response'),
  (0.0049079577, 'memory'),
  (0.0046701324, 'orientation'),
  (0.004652298, 'map'),
  (0.0044042952, 'activity'),
  (0.0042442977, 'cortex'),
  (0.0037213287, 'receptive'),
  (0.0034765801, 'location'),
  (0.0034255898, 'receptive_field'),
  (0.0033544719, 'connection'),
  (0.0032896711, 'cortical'),
  (0.0032853843, 'neuron'),
  (0.0032728878, 'position'),
  (0.0030441082, 'direction')],
 -0.9576207793821232)

In [25]:
model.inference([corpus[0]])[0]

array([[1.9803188e+01, 6.7137939e+02, 4.3383028e-02, 5.6552000e-02,
        4.8746493e-02, 1.0835744e+02, 4.8054527e-02, 3.3769492e-02,
        5.6352612e-02, 4.5650158e-02]], dtype=float32)

### Perplexity evaluation
We want to assess the model with something more convincing than just looking at topic profiles and document profiles. This is necessary for the possibility of comparing different models, for example, those obtained with different run parameters. Let's learn to measure **perplexity**. The function `model.state.get_lambda` returns the unnormalized $\Phi$ matrix, and `model.inference` estimates the unnormalized $\Theta$ matrix for a list of documents.

We iterate through the collection and calculate perplexity using the formula. The lower the perplexity, the better.

In [26]:
def perplexity(model, corpus):
    corpus_length = 0
    log_likelihood = 0
    topic_profiles = model.state.get_lambda() / np.sum(model.state.get_lambda(), axis=1)[:, np.newaxis]
    for document in corpus:
        gamma, _ = model.inference([document])
        document_profile = gamma / np.sum(gamma)
        for term_id, term_count in document:
            corpus_length += term_count
            term_probability = np.dot(document_profile, topic_profiles[:, term_id])
            log_likelihood += term_count * log(term_probability.item())
    perplexity = np.exp(-log_likelihood / corpus_length)
    return perplexity

In [27]:
print('Perplexity: {}'.format(perplexity(model, corpus)))

Perplexity: 2792.500048725142


In [28]:
model_5 = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=5,
    passes=epochs,
    eval_every=eval_every
)
model_20 = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=20,
    passes=epochs,
    eval_every=eval_every
)

2025-04-01 14:03:33,855 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2025-04-01 14:03:33,856 : INFO : using serial LDA version on this node
2025-04-01 14:03:33,857 : INFO : running online (multi-pass) LDA training, 5 topics, 5 passes over the supplied corpus of 1740 documents, updating model once every 1740 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2025-04-01 14:03:33,858 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2025-04-01 14:03:33,858 : INFO : PROGRESS: pass 0, at document #1740/1740
2025-04-01 14:03:38,722 : INFO : optimized alpha [0.10357928, 0.13110664, 0.22551106, 0.111403726, 0.15937737]
2025-04-01 14:03:38,724 : INFO : topic #0 (0.104): 0.007*"neuron" + 0.005*"memory" + 0.004*"layer" + 0.004*"cell" + 0.003*"hidden" + 0.003*"control" + 0.003*"signal" + 0.003*"net" + 0.003*"map" + 0.003*"circuit"
20

In [29]:
print('Perplexity 5: {}'.format(perplexity(model_5, corpus)))
print('Perplexity 20: {}'.format(perplexity(model_20, corpus)))

Perplexity 5: 3121.8680595351993
Perplexity 20: 2518.910892162995


### Word2Vec model
Word2Vec is one of the fundamental neural network models of the "pre-transformer" era (2013-2018). The essence of the model is to build a mapping of words into an $N$-dimensional space (embeddings) with certain characteristics. Two words have more similar embeddings the more similar the contexts in which they are used.

In the `gensim` library, two methods for building word2vec are implemented:
  - Skip-grams (SG)
  - Continuous-bag-of-words (CBOW)

## Demo
For the demonstration, let's take a pre-trained model trained on the Google News dataset, containing approximately 3 million English words and phrases.

In [30]:
# download the model ~1.6GB 
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

2025-04-01 14:05:19,983 : INFO : loading projection weights from /Users/Aleksandr.Avdiushenko/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2025-04-01 14:05:32,568 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/Aleksandr.Avdiushenko/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-04-01T14:05:32.568123', 'gensim': '4.3.3', 'python': '3.12.6 (v3.12.6:a4a2d2b0d85, Sep  6 2024, 16:08:03) [Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-15.3.2-arm64-arm-64bit', 'event': 'load_word2vec_format'}


In [31]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key )} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [32]:
vec_king = wv['king']
print(vec_king[:10])

[ 0.12597656  0.02978516  0.00860596  0.13964844 -0.02563477 -0.03613281
  0.11181641 -0.19824219  0.05126953  0.36328125]


Using the model, you can compute the distances between words.

In [33]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


You can also find the most similar words to a given one.

In [34]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

[('SUV', 0.8532192707061768), ('vehicle', 0.8175783753395081), ('pickup_truck', 0.7763689756393433), ('Jeep', 0.7567334175109863), ('Ford_Explorer', 0.7565719485282898)]


In [35]:
vec_example = wv['king'] - wv['man'] + wv['woman']

similars = wv.most_similar(positive=[vec_example])
print(similars)

[('king', 0.8449392318725586), ('queen', 0.7300516366958618), ('monarch', 0.6454660296440125), ('princess', 0.6156251430511475), ('crown_prince', 0.5818676948547363), ('prince', 0.5777117609977722), ('kings', 0.5613663792610168), ('sultan', 0.5376776456832886), ('Queen_Consort', 0.5344247817993164), ('queens', 0.5289887189865112)]


In [36]:
vec_example = wv['programmer'] - wv['man'] + wv['woman'] 

similars = wv.most_similar(positive=[vec_example])
print(similars)

[('programmer', 0.885962188243866), ('programmers', 0.6040860414505005), ('computer_programmer', 0.5623369216918945), ('coder', 0.5616979598999023), ('Programmer', 0.5576066374778748), ('programer', 0.5161396265029907), ('graphic_designer', 0.5139066576957703), ('coders', 0.48765403032302856), ('designer', 0.4822673797607422), ('librarian', 0.4649229943752289)]
